# Predicting 🙊Toxic comments with Lightning⚡Flash

This is just a warpper for the original kernel to follow offline rules...

**see: https://www.kaggle.com/jirkaborovec/toxic-comments-with-lightning-flash**

In [ ]:
! pip install -q lightning-flash[text] --no-index --find-links /kaggle/input/toxic-comments-with-lightning-flash/frozen_packages
# ! pip install -q 'https://github.com/PyTorchLightning/lightning-flash/archive/refs/heads/master.zip#egg=lightning-flash[tabular]'
# ! pip install -q --upgrade matplotlib pandas --force-reinstall --no-index --find-links /kaggle/input/packages
! pip list | grep -E "lightning|torch|transformer"

In [ ]:
! ls -l /kaggle/input/
! ls -l /kaggle/input/jigsaw-toxic-severity-rating
! ls -l /kaggle/input/toxic-comments-with-lightning-flash

In [ ]:
# ! ls -l /kaggle/input/toxic-comments-with-lightning-flash/cache/huggingface
# ! mkdir -p ~/.cache/huggingface/
# ! rsync -ahv /kaggle/input/toxic-comments-with-lightning-flash/cache/huggingface/ ~/.cache/huggingface
# ! ls -l ~/.cache/huggingface

In [ ]:
import pandas as pd

csv_comemnts = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"
df_comments = pd.read_csv(csv_comemnts, index_col="comment_id")
display(df_comments.head())

## Classify new comments with Flash Lightning
See the classification docs: https://lightning-flash.readthedocs.io/en/stable/reference/text_classification.html

In [ ]:
from flash.text import TextClassifier

# m2 = TextClassifier.load_from_checkpoint("text_classification_model.pt", backbone="./my_named_bert")
model = TextClassifier.load_from_checkpoint(
    "/kaggle/input/toxic-comments-with-lightning-flash/text_classification_model.pt",
    backbone="/kaggle/input/huggingface-roberta-variants/pytorch-xlm-roberta-base/pytorch-xlm-roberta-base",
    # local_files_only=True,
    num_classes=2,
)

In [ ]:
import math
from tqdm.auto import tqdm
from flash.core.classification import Logits, Probabilities

model.output = Logits()
batch_size = 512

predictions = []
for i in tqdm(range(math.ceil(len(df_comments) / batch_size))):
    batch = df_comments["text"][i * batch_size:(i + 1) * batch_size]
    predictions += model.predict(batch)

print(f"inputs={len(df_comments)} ; preds={len(predictions)}")
print(predictions[0])

In [ ]:
import numpy as np
# import matplotlib.pyplot as plt

predictions = np.array(predictions)[:, -1]
# _= pd.Series(predictions).plot.hist(predictions, bins=20, grid=True)

In [ ]:
import os

df_comments["score"] = predictions
try:
    df_comments["score"].to_csv("submission.csv")
except:
    lines = ["comment_id,score"]
    lines += [f"{idx},{row['score']}" for idx, row in df_comments[["score"]].iterrows()]
    with open("submission.csv", "w") as fp:
        fp.write(os.linesep.join(lines))
    # print(lines[:5])

! head submission.csv